In [1]:
import pandas as pd
import folium
import pandas as pd
import datetime
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [2]:
yuco = pd.read_csv('data_cleaned/yuco.csv')
quinsy = pd.read_csv('data_cleaned/quinsy.csv')

In [3]:
path_yuco = []
step = 100
for lon, lat in zip(yuco['INX Longitude (�)'][::step], yuco['INX Latitude (�)'][::step]):
    path_yuco.append((lat, lon))

In [4]:
m = folium.Map(location=(quinsy['A-Frame_Extended Latitude'].mean(), quinsy['A-Frame_Extended Longitude'].mean()),
               zoom_start=15, control_scale = True)
folium.PolyLine(path_yuco, tooltip="YUCO").add_to(m)
for i, row in quinsy.iterrows():
    folium.Marker(
    location=[row['A-Frame_Extended Latitude'], row['A-Frame_Extended Longitude']],
    tooltip=row['[Mainline] Name'],
    #popup="Timberline Lodge",
    icon=folium.Icon(color="green"),
    ).add_to(m)

m

# CTD data

In [5]:
ctd_files = list(Path('data_cleaned/ctd/csv').glob('*.csv'))
ctd_dict = {}
for fn in ctd_files:
    ctd_name = fn.name.split('.')[0]
    ctd_dict[ctd_name] = pd.read_csv(fn, parse_dates=['datetime'])

In [6]:
for name, df in ctd_dict.items():
    plot_vars =  [
     'Temp [°C]',
     'SALIN [PSU]',
    # 'sat [%]',
     'DO_mg [mg/l]',
     'Chl_A [µg/l]',]
    fig, axs = plt.subplots(2,2, figsize=(10, 10))
    axs = axs.ravel()
    for i, var in enumerate(plot_vars):
        ax = axs[i]
        sub = df[df['Press [dbar]']>1][var]
        interquart = np.nanpercentile(sub, 75) -  np.nanpercentile(sub, 25)
        lims = [np.nanpercentile(sub, 1) - interquart * 0.1, np.nanpercentile(sub, 99) + interquart * 0.1]
        max_pressure_index = np.argmax(df['Press [dbar]'])
        df_down = df[df.index <= max_pressure_index]
        df_up = df[df.index > max_pressure_index]
        ax.scatter(df_down[var], df_down['Press [dbar]'], label=var + " downcast", s=1)
        ax.scatter(df_up[var], df_up['Press [dbar]'],label=var + " upcast", s=1)
        ax.invert_yaxis()
        ax.set(xlim=lims)
        ax.legend()
    axs[0].set(ylabel="pressure (dbar)")
    axs[2].set(ylabel="pressure (dbar)")
    fig.suptitle(name)
    fig.savefig(f'figures/ctd/profiles/{name}.png')
    plt.close('all')

In [9]:
ds = xr.open_dataset('data_cleaned/ctd/nc/ctd_gridded.nc')

In [10]:
for var in list(ds):
    if var == 'datetime':
        continue
    ds[var].plot()
    plt.gca().invert_yaxis()
    plt.savefig(f'figures/ctd/gridded/{var}.png')
    plt.close('all')